In [1]:
!pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 2.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 729.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 468.0 kB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from sklearn.metrics import classification_report, confusion_matrix
from transformers import AutoTokenizer
import re

ModuleNotFoundError: No module named 'pandas'

In [10]:
try:
    from wordcloud import WordCloud
    HAS_WORDCLOUD = True
except ImportError:
    HAS_WORDCLOUD = False
    print("Warning: 'wordcloud' library not found. Install via 'pip install wordcloud' for visual insights.")

In [23]:
plt.style.use('ggplot')

def generate_analytics_report(csv_path, output_dir="./report"):
    """
    Генерирует отчет с метриками и графиками (без облака слов).
    Все подписи на русском языке.
    """
    print(f"--- Генерация отчета для {csv_path} ---")
    os.makedirs(output_dir, exist_ok=True)

    # Читаем данные
    try:
        df = pd.read_csv(csv_path)
    except:
        df = pd.read_csv(csv_path, sep=';')

    if 'predicted_label' not in df.columns:
        raise ValueError("ОШИБКА: В CSV нет колонки 'predicted_label'.")

    # Приводим к int
    df['predicted_label'] = pd.to_numeric(df['predicted_label'], errors='coerce').fillna(-1).astype(int)

    report_data = {'business_metrics': {}, 'images': {}}

    classes_map = {0: 'Нейтрально', 1: 'Позитив', 2: 'Негатив'}

    # 1. МЕТРИКИ КАЧЕСТВА (F1, Матрица ошибок)
    if 'label' in df.columns:
        print(">>> Считаем метрики качества")
        y_true = pd.to_numeric(df['label'], errors='coerce').fillna(-1).astype(int)
        y_pred = df['predicted_label']

        # Убираем мусор (-1)
        mask = (y_true != -1) & (y_pred != -1)
        y_true_clean = y_true[mask]
        y_pred_clean = y_pred[mask]

        if len(y_true_clean) > 0:
            # JSON Метрики (F1)
            cls_report = classification_report(y_true_clean, y_pred_clean, output_dict=True, zero_division=0)
            report_data['quality_metrics'] = cls_report

            # ГРАФИК: Матрица ошибок
            cm = confusion_matrix(y_true_clean, y_pred_clean)
            plt.figure(figsize=(8, 6))

            # Подписи для осей (берем из словаря, чтобы порядок был 0,1,2)
            tick_labels = [classes_map[i] for i in sorted(classes_map.keys())]

            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', annot_kws={"size": 14},
                        xticklabels=tick_labels,
                        yticklabels=tick_labels)

            plt.title("Матрица ошибок (Confusion Matrix)", fontsize=14, pad=20)
            plt.xlabel("Предсказание модели", fontsize=12, fontweight='bold')
            plt.ylabel("На самом деле", fontsize=12, fontweight='bold')

            plt.figtext(0.5, -0.05, "По диагонали (темные квадраты) — правильные ответы.\nВне диагонали — ошибки модели.",
                        wrap=True, horizontalalignment='center', fontsize=10, color='gray')

            cm_path = os.path.join(output_dir, "confusion_matrix.png")
            plt.savefig(cm_path, bbox_inches='tight')
            plt.close()
            report_data['images']["confusion_matrix"] = cm_path

    # 2. БИЗНЕС-МЕТРИКИ (Распределение, Индекс)
    print(">>> Считаем бизнес-статистику")

    n_neu = len(df[df['predicted_label'] == 0])
    n_pos = len(df[df['predicted_label'] == 1])
    n_neg = len(df[df['predicted_label'] == 2])
    total = len(df)

    if total > 0:
        # Индекс настроения: (Позитив - Негатив) / Всего
        sentiment_index = (n_pos - n_neg) / total

        report_data['business_metrics'] = {
            'sentiment_index': round(sentiment_index, 4),
            'total_count': total,
            'counts': {'Neutral': n_neu, 'Positive': n_pos, 'Negative': n_neg}
        }

        # ГРАФИК: Pie Chart
        plt.figure(figsize=(7, 7))

        labels = [f'Нейтрально ({n_neu})', f'Позитив ({n_pos})', f'Негатив ({n_neg})']
        sizes = [n_neu, n_pos, n_neg]
        colors = ['#bdc3c7', '#2ecc71', '#e74c3c'] # Серый, Зеленый, Красный

        plt.pie(sizes, labels=labels, autopct='%1.1f%%',
                colors=colors, startangle=90, pctdistance=0.85)

        centre_circle = plt.Circle((0,0), 0.70, fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)

        plt.title(f"Распределение тональности\nИндекс лояльности: {sentiment_index:.2f}", fontsize=14)

        pie_path = os.path.join(output_dir, "sentiment_pie.png")
        plt.savefig(pie_path, bbox_inches='tight')
        plt.close()
        report_data['images']["sentiment_pie"] = pie_path

    # Сохраняем JSON
    json_path = os.path.join(output_dir, "final_report.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(report_data, f, indent=4, ensure_ascii=False)

    print(f"Отчет готов: {json_path}")
    return json_path

In [21]:
if __name__ == "__main__":
    # 1. Сначала делаем инференс (если еще не сделали)

    # 2. Потом генерируем отчет

    CSV_FILE = "/content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/data/processed/test_predictions.csv"
    MODEL_DIR = "/content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/models/cointegrated_ART"

    if os.path.exists(CSV_FILE):
        generate_analytics_report(CSV_FILE, output_dir="/content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/data/report/run")


--- Генерация отчета для /content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/data/processed/train_predictions.csv ---
>>> Считаем метрики качества...
>>> Считаем бизнес-статистику
Отчет готов: /content/drive/MyDrive/Colab Notebooks/Hack_28-11-2025/data/report/test_run_2/final_report.json
